In [7]:
import pandas as pd
import datetime as dt
import altair as alt

In [8]:
# read in json (converts to df)
spotify_data = pd.read_json("StreamingHistory1.json")

# set df as datetime
spotify_data['endTime'] = pd.to_datetime(spotify_data['endTime'])

# convert from milliseconds to seconds (for readibility)
spotify_data['seconds_played'] = spotify_data['msPlayed']/1000

In [9]:
long_listens = spotify_data[spotify_data['seconds_played'] >= 120]

# floor endtime by date
long_listens['endTime'] = long_listens['endTime'].dt.round('D') 

# floor endtime by month
long_listens['endTime'] = long_listens['endTime'].dt.to_period('M').dt.to_timestamp()

# group by endtime and artist, take sum of seconds played
long_listens = long_listens[['endTime', 'artistName', 'seconds_played']].groupby(['endTime', 'artistName'], as_index = False)['seconds_played'].agg('sum')

# keep top 5 listened artists in a period
long_listens = long_listens.sort_values('seconds_played').groupby(['endTime']).tail(5)

/Users/ester/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/ester/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
long_listens

,endTime,artistName,seconds_played
2,2020-04-01,Demi Lovato,203.672
4,2020-04-01,Japanese Breakfast,213.986
5,2020-04-01,Rex Orange County,267.520
0,2020-04-01,Beach Bunny,1016.019
366,2020-09-01,Beach Bunny,1294.022
...,...,...,...
855,2021-01-01,Taylor Swift,25138.460
697,2020-12-01,Pentatonix,25467.641
1171,2021-03-01,Lizzy McAlpine,26944.917
1317,2021-04-01,Lizzy McAlpine,66648.158


In [11]:
# make the chart
alt.data_transformers.disable_max_rows()

chart = alt.Chart(long_listens).mark_area().encode(
    x = 'endTime',
    y = 'seconds_played',
    color = 'artistName'
).properties(
    height = 400,
    width = 800,
)


text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=7
).encode(
    text='artistName'
)

text + chart


alt.LayerChart(...)